In [1]:
import time
import pandas as pd
import mel_functions as zl

In [2]:
st = zl.zipcodes_list(st_items = ['60647', '60610', '60614', '60622'])

driver = zl.init_driver("/Users/swagbird/Desktop/chromedriver2")

zl.navigate_to_website(driver, "https://www.zillow.com/homes/for_rent/3-_beds/3-_baths/")

num_search_terms = len(st)

output_data = []

In [3]:
for idx, term in enumerate(st):
    # Enter search term and execute search.
    if zl.enter_search_term(driver, term):
        print("Entering search term %s of %s" % 
              (str(idx + 1), str(num_search_terms)))
    else:
        print("Search term %s failed, moving on to next search term\n***" % 
              str(idx + 1))
        continue

    # Check to see if any results were returned from the search.
    # If there were none, move onto the next search.
    if zl.test_for_no_results(driver):
        print("Search %s returned zero results. Moving on to next search\n***" %
              str(term))
        continue
        
    # Pull the html for each page of search results. Zillow caps results at
    # 20 pages, each page can contain 25 home listings, thus the cap on home
    # listings per search is 500
    raw_data = zl.get_html(driver)
    print("%s pages of listings found" % str(len(raw_data)))

    # Take the extracted HTML and split it up by individual home listings.
    listings = zl.get_listings(raw_data)
    print("%s home listings scraped\n***" % str(len(listings)))

    for home in listings:
        new_obs = []
        parser = zl.html_parser(home)
        
        # Street Address
        new_obs.append(parser.get_street_address())

        # City
        new_obs.append(parser.get_city())

        # State
        new_obs.append(parser.get_state())

        # Zipcode
        new_obs.append(parser.get_zipcode())

        # Price
        new_obs.append(parser.get_price())
        
        # Sqft
        new_obs.append(parser.get_sqft())
        
        # Bedrooms
        new_obs.append(parser.get_bedrooms())
        
        # Bathrooms
        new_obs.append(parser.get_bathrooms())
        
        # Days on the Market/Zillow
        new_obs.append(parser.get_days_on_market())
        
        # Sale Type (House for Sale, New Construction, Foreclosure, etc.)
        new_obs.append(parser.get_sale_type())
        
        # URL for each house listing
        new_obs.append(parser.get_url())
        
        # Append new_obs to list output_data.
        output_data.append(new_obs)

    # Close the webdriver connection.
    zl.close_connection(driver)

Entering search term 1 of 4
1 pages of listings found
15 home listings scraped
***


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=62032): Max retries exceeded with url: /session/3bff2c63d75466934aba86f8322ac755/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11753b400>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
# Write data to data frame, then to CSV file.
file_name = "%s_%s.csv" % (str(time.strftime("%Y-%m-%d")),
                          str(time.strftime("%H%M%S")))
columns = ["address", "city", "state", "zip", "price", "sqft", "bedrooms",
           "bathrooms", "days_on_zillow", "sale_type", "url"]
pd.DataFrame(output_data, columns=columns).drop_duplicates().to_csv(
    file_name, index = False, encoding = "UTF-8"
)